In [49]:
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyOAuth

scope='user-library-read playlist-modify-public playlist-modify-private'


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id="6584944ee0d9495480193d9997a9efb7",
                                               client_secret="ac4205f012b64179acd46c0fbdb33f36",
                                               redirect_uri="http://127.0.0.1:5000/redirect",
                                               scope=scope))

songs = []
pl = sp.current_user_playlists()['items'];
id = pl[0]['id']
playlist = sp.playlist_tracks(id)

tracks_data = []

for track in playlist['items']:
    
    track_uri = track['track']['uri']
    track_info = sp.track(track_uri)
    track_popularity = track_info['popularity']
    
    artist_uri = track['track']['artists'][0]['uri']
    artist = sp.artist(artist_uri)
    track_data = {
            'artist_name': track['track']['artists'][0]['name'],
            'id': track['track']['id'],
            'track_name': track['track']['name'],
            'danceability': None,
            'energy': None,
            'key': None,
            'loudness': None,
            'mode': None,
            'speechiness': None,
            'acousticness': None,
            'instrumentalness': None,
            'liveness': None,
            'valence': None,
            'tempo': None,
            'artist_pop': artist['popularity'],
            'genres': artist['genres'],
            'track_pop': track_popularity
        }
    
    audio_features = sp.audio_features([track['track']['uri']])[0]
    if audio_features:
        track_data['danceability'] = audio_features['danceability']
        track_data['energy'] = audio_features['energy']
        track_data['key'] = audio_features['key']
        track_data['loudness'] = audio_features['loudness']
        track_data['mode'] = audio_features['mode']
        track_data['speechiness'] = audio_features['speechiness']
        track_data['acousticness'] = audio_features['acousticness']
        track_data['instrumentalness'] = audio_features['instrumentalness']
        track_data['liveness'] = audio_features['liveness']
        track_data['valence'] = audio_features['valence']
        track_data['tempo'] = audio_features['tempo']
        
    tracks_data.append(track_data)

songs_df = pd.DataFrame(tracks_data)

print(songs_df)

             artist_name                      id               track_name  \
0                Nirvana  4CeeEOM32jQcH3eN9Q2dGj  Smells Like Teen Spirit   
1                   Blur  1FTSo4v6BOZH9QxKc3MbVM   Song 2 - 2012 Remaster   
2  Red Hot Chili Peppers  48UPSzbZjgc449aqz8bxox          Californication   
3        Franz Ferdinand  20I8RduZC2PWMWTDCZuuAN              Take Me Out   
4        The Cranberries  7EZC6E7UjZe63f1jRmkWxt                   Zombie   

   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.502   0.912    1    -4.556     1       0.0564      0.000025   
1         0.674   0.789    8    -6.903     1       0.0676      0.001780   
2         0.592   0.767    9    -2.788     0       0.0270      0.002100   
3         0.277   0.663    4    -8.821     0       0.0377      0.000409   
4         0.309   0.649    7    -8.994     1       0.0350      0.015800   

   instrumentalness  liveness  valence    tempo  artist_pop  \
0          0.000173    